# Survey evaluation

In [1]:
import re
from os.path import join
import numpy as np
import pandas as pd
from sklearn.metrics import jaccard_similarity_score
from constants import DATA_BASE, LDA_PATH, TPX_PATH, ETL_PATH
from utils import load, Unlemmatizer, TopicsLoader

pd.options.display.max_rows = 200
pd.options.display.max_columns = 300

-----

### load data

In [2]:
# GET DEWAC1 TOP 20 TOPIC TERMS

dataset = 'dewac1'
param_ids = 'e42'
nbs_topics = 100
version = 'noun'
topn = 20

topics_loader = TopicsLoader(
    dataset=dataset,
    param_ids=param_ids,
    nbs_topics=nbs_topics,
    version=version,
    topn=topn
)
tpx = topics_loader.topics
tpx

Loading dictionary from ../data/preprocessed/LDAmodel/noun/bow/dewac1_noun_bow.dict
Loading model from ../data/preprocessed/LDAmodel/noun/bow/e42/dewac1_LDAmodel_e42_100_ep30


term0  \
dataset param_id nb_topics topic_idx                        
dewac1  e42      100       0                       Partei   
                           1                        König   
                           2          Lebenspartnerschaft   
                           3                       BetrVG   
                           4                 Gesellschaft   
                           5                      Artikel   
                           6                       Kirche   
                           7                 Entscheidung   
                           8                      Richter   
                           9                  Zivildienst   
                           10                      Mensch   
                           11                      Physik   
                           12                       Musik   
                           13                     Theater   
                           14                 Unternehmen   
                           15                       Regie   
                           16                         SPD   
                           17                Rechtsanwalt   
                           18                       Leute   
                           19                  Richtlinie   
                           20                   Bewerbung   
                           21                       Spiel   
                           22                        Frau   
                           23                         Dr.   
                           24                        Tier   
                           25                      Mensch   
                           26                        Land   
                           27                        Wahl   
                           28                        Ring   
                           29                 Hausaufgabe   
                           30                 Deutschland   
                           31                       Seite   
                           32                         USA   
                           33                        Bild   
                           34                      Israel   
                           35                    Programm   
                           36                     Kollege   
                           37                     Hamburg   
                           38                      Mensch   
                           39                  Mathematik   
                           40                        Kind   
                           41                    Training   
                           42                         Art   
                           43         Nachhilfeunterricht   
                           44                       Organ   
                           45                         Uhr   
                           46                     Sendung   
                           47                        Witz   
                           48                        Abs.   
                           49                Arbeitsplatz   
                           50                      Mensch   
                           51                        Euro   
                           52                    Hochzeit   
                           53                  Österreich   
                           54                      Berlin   
                           55                   Kommentar   
                           56               Veranstaltung   
                           57                     Störung   
                           58                    Akademie   
                           59                         Tag   
                           60                  Ausbildung   
                           61                    Fahrzeug   
                           62                   Regierung   
                           63                     Beitrag   
                          

In [3]:
file = join(DATA_BASE, 'crowdsourcing', 'TopicCoherenceSurvey', 'final_data', 'data_topicsde_2018-12-25_14-46.csv')
results = pd.read_csv(file, delimiter='\t', header=[0, 1])
results

,CASE,SERIAL,REF,QUESTNNR,MODE,STARTED,T001,T001_01,T001_02,T001_03,T001_04,T001_05,T001_06,T001_07,T001_08,T001_09,T001_10,T001_11,T001_12,T001_13,T001_14,T001_15,T001_16,T001_17,T001_18,T001_19,T001_20,T002,T002_01,T002_02,T002_03,T002_04,T002_05,T002_06,T002_07,T002_08,T002_09,T002_10,T002_11,T002_12,T002_13,T002_14,T002_15,T002_16,T002_17,T002_18,T002_19,T002_20,T003,T003_01,T003_02,T003_03,T003_04,T003_05,T003_06,T003_07,T003_08,T003_09,T003_10,T003_11,T003_12,T003_13,T003_14,T003_15,T003_16,T003_17,T003_18,T003_19,T003_20,T004,T004_01,T004_02,T004_03,T004_04,T004_05,T004_06,T004_07,T004_08,T004_09,T004_10,T004_11,T004_12,T004_13,T004_14,T004_15,T004_16,T004_17,T004_18,T004_19,T004_20,T005,T005_01,T005_02,T005_03,T005_04,T005_05,T005_06,T005_07,T005_08,T005_09,T005_10,T005_11,T005_12,T005_13,T005_14,T005_15,T005_16,T005_17,T005_18,T005_19,T005_20,T006,T006_01,T006_02,T006_03,T006_04,T006_05,T006_06,T006_07,T006_08,T006_09,T006_10,T006_11,T006_12,T006_13,T006_14,T006_15,T006_16,T006_17,T006_18,T006_19,T006_20,T007,T007_01,T007_02,T007_03,T007_04,T007_05,T007_06,T007_07,T007_08,T007_09,T007_10,T007_11,T007_12,T007_13,T007_14,T007_15,T007_16,T007_17,...,T909_17,T909_18,T909_19,T909_20,T910,T910_01,T910_02,T910_03,T910_04,T910_05,T910_06,T910_07,T910_08,T910_09,T910_10,T910_11,T910_12,T910_13,T910_14,T910_15,T910_16,T910_17,T910_18,T910_19,T910_20,L001_01,L002_01,L003_01,L004_01,L005_01,L006_01,L007_01,L008_01,L009_01,L010_01,L101_01,L102_01,L103_01,L104_01,L105_01,L106_01,L107_01,L108_01,L109_01,L110_01,L201_01,L202_01,L203_01,L204_01,L205_01,L206_01,L207_01,L208_01,L209_01,L210_01,L301_01,L302_01,L303_01,L304_01,L305_01,L306_01,L307_01,L308_01,L309_01,L310_01,L401_01,L402_01,L403_01,L404_01,L405_01,L406_01,L407_01,L408_01,L409_01,L410_01,L501_01,L502_01,L503_01,L504_01,L505_01,L506_01,L507_01,L508_01,L509_01,L510_01,L601_01,L602_01,L603_01,L604_01,L605_01,L606_01,L607_01,L608_01,L609_01,L610_01,L701_01,L702_01,L703_01,L704_01,L705_01,L706_01,L707_01,L708_01,L709_01,L710_01,L801_01,L802_01,L803_01,L804_01,L805_01,L806_01,L807_01,L808_01,L809_01,L810_01,L901_01,L902_01,L903_01,L904_01,L905_01,L906_01,L907_01,L908_01,L909_01,L910_01,DX03,TIME001,TIME002,TIME003,TIME004,TIME005,TIME006,TIME007,TIME008,TIME009,TIME010,TIME011,TIME012,TIME013,TIME_SUM,MAILSENT,LASTDATA,FINISHED,Q_VIEWER,LASTPAGE,MAXPAGE,MISSING,MISSREL,TIME_RSI,DEG_TIME
,Interview-Nummer (fortlaufend),Seriennummer (sofern verwendet),Referenz (sofern im Link angegeben),"Fragebogen, der im Interview verwendet wurde",Interview-Modus,Zeitpunkt zu dem das Interview begonnen hat,Topic 1: Ausweichoption (negativ) oder Anzahl ausgewählter Optionen,Topic 1: Partei,Topic 1: PDS,Topic 1: Arbeiter,Topic 1: Kampf,Topic 1: Sozialismus,Topic 1: DDR,Topic 1: Kommunisten,Topic 1: Gewerkschaften,Topic 1: Kapitalismus,Topic 1: Linke,Topic 1: Bewegung,Topic 1: Genossen,Topic 1: Kraft,Topic 1: Revolution,Topic 1: Marx,Topic 1: Kapital,Topic 1: Politik,Topic 1: Arbeiterklasse,Topic 1: Homepage,Topic 1: Stalin,Topic 2: Ausweichoption (negativ) oder Anzahl ausgewählter Optionen,Topic 2: König,Topic 2: Frau,Topic 2: Mädchen,Topic 2: Mann,Topic 2: Wasser,Topic 2: Märchen,Topic 2: Hand,Topic 2: Tag,Topic 2: Schiff,Topic 2: Vater,Topic 2: Bruder,Topic 2: Baum,Topic 2: Kinder,Topic 2: Haus,Topic 2: Mutter,Topic 2: Wald,Topic 2: Tochter,Topic 2: Schwester,Topic 2: Prinzessin,Topic 2: Nacht,Topic 3: Ausweichoption (negativ) oder Anzahl ausgewählter Optionen,Topic 3: Lebenspartnerschaft,Topic 3: Lebenspartner,Topic 3: Ehe,Topic 3: Schwule,Topic 3: Partner,Topic 3: Lesben,Topic 3: Ehegatten,Topic 3: Abs.,Topic 3: Paare,Topic 3: Diskriminierung,Topic 3: Begründung,Topic 3: Partnerschaft,Topic 3: Recht,Topic 3: Lebensgemeinschaft,Topic 3: Lebenspartnerschaftsgesetz,Topic 3: Gesetz,Topic 3: Personen,Topic 3: Homosexuelle,Topic 3: Erklärung,Topic 3: Homosexualität,Topic 4: Ausweichoption (negativ) oder Anzahl ausgewählter Optionen,Topic 4: BetrVG,Topic 4: Betriebsrat,Topic 4: Abs.,Topic 4: Nr.,Topic

-----

### evaluate annotator's correlation

and select annotators

In [4]:
pattern = re.compile(r'^T\d\d\d_\d\d')
cols = [t for t in results.columns if pattern.match(t[0])]
topics = [t[0].split('_')[0] for t in cols]
res = (results[cols].T - 1).astype(bool)
res['topic'] = topics
res = res.set_index('topic', append=True)
res

,,,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
,,topic,,,,,,,,,,,,,,,,,
T001_01,Topic 1: Partei,T001,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
T001_02,Topic 1: PDS,T001,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False
T001_03,Topic 1: Arbeiter,T001,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
T001_04,Topic 1: Kampf,T001,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False
T001_05,Topic 1: Sozialismus,T001,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
T001_06,Topic 1: DDR,T001,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
T001_07,Topic 1: Kommunisten,T001,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
T001_08,Topic 1: Gewerkschaften,T001,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
T001_09,Topic 1: Kapitalismus,T001,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [5]:
bad_raters = [0, 1, 10, 12, 8, 2, 3]

In [6]:
def aggregate_scores(grp):
    return grp.T.sum(axis=1)

ratings = res.groupby('topic').apply(aggregate_scores)
ratings = ratings.drop(bad_raters, axis=1)
rate_corr = ratings.corr('pearson')
mask = rate_corr == 1.
rate_corr[mask.values] = np.nan
print(f'Avg correlation among all raters and topics: {rate_corr.mean().mean():.2f}')
rate_corr

Avg correlation among all raters and topics: 0.57


,4,5,6,7,9,11,13,14,15,16
4,NaN,0.366542,0.728157,0.671020,0.614971,0.666593,0.526248,0.615414,0.652402,0.479563
5,0.366542,NaN,0.458720,0.523207,0.492539,0.482890,0.495081,0.418695,0.527168,0.597667
6,0.728157,0.458720,NaN,0.708546,0.581072,0.631821,0.606728,0.673422,0.622195,0.574950
7,0.671020,0.523207,0.708546,NaN,0.584631,0.673715,0.636487,0.661888,0.712371,0.603491
9,0.614971,0.492539,0.581072,0.584631,NaN,0.549924,0.494238,0.502135,0.551304,0.443531
11,0.666593,0.482890,0.631821,0.673715,0.549924,NaN,0.534131,0.586308,0.602455,0.536915
13,0.526248,0.495081,0.606728,0.636487,0.494238,0.534131,NaN,0.643701,0.561303,0.429262
14,0.615414,0.418695,0.673422,0.661888,0.502135,0.586308,0.643701,NaN,0.564594,0.499119
15,0.652402,0.527168,0.622195,0.712371,0.551304,0.602455,0.561303,0.564594,NaN,0.500295
16,0.479563,0.597667,0.574950,0.603491,0.443531,0.536915,0.429262,0.499119,0.500295,NaN


In [7]:
rate_corr.mean().sort_values()

5     0.484723
16    0.518310
9     0.534927
13    0.547464
14    0.573920
11    0.584972
15    0.588232
4     0.591212
6     0.620623
7     0.641706
dtype: float64

In [8]:
annotators = [c for c in ratings.columns]
ratings['avg'] = ratings.mean(axis=1)
ratings['score'] = (15 - ratings['avg']) / 14
ratings

,4,5,6,7,9,11,13,14,15,16,avg,score
topic,,,,,,,,,,,,
T001,5,2,2,2,3,1,2,2,3,1,2.3,0.907143
T002,9,8,3,3,9,9,2,9,7,9,6.8,0.585714
T003,5,3,2,4,7,3,3,4,9,3,4.3,0.764286
T004,6,7,2,7,7,8,5,9,13,5,6.9,0.578571
T005,10,4,4,5,10,8,3,4,7,6,6.1,0.635714
T006,9,5,1,2,8,10,2,2,10,4,5.3,0.692857
T007,3,1,2,2,3,4,2,1,3,2,2.3,0.907143
T008,6,3,2,1,3,4,2,1,4,2,2.8,0.871429
T009,4,6,5,6,13,9,4,6,6,7,6.6,0.600000


In [12]:
# store survey topics with gold scores
scoredtopics = tpx.copy()
scoredtopics['Gold'] = ratings.score.values
scoredtopics.to_csv(join(TPX_PATH, 'survey_topics.csv'))
scoredtopics

term0  \
dataset param_id nb_topics topic_idx                        
dewac1  e42      100       0                       Partei   
                           1                        König   
                           2          Lebenspartnerschaft   
                           3                       BetrVG   
                           4                 Gesellschaft   
                           5                      Artikel   
                           6                       Kirche   
                           7                 Entscheidung   
                           8                      Richter   
                           9                  Zivildienst   
                           10                      Mensch   
                           11                      Physik   
                           12                       Musik   
                           13                     Theater   
                           14                 Unternehmen   
                           15                       Regie   
                           16                         SPD   
                           17                Rechtsanwalt   
                           18                       Leute   
                           19                  Richtlinie   
                           20                   Bewerbung   
                           21                       Spiel   
                           22                        Frau   
                           23                         Dr.   
                           24                        Tier   
                           25                      Mensch   
                           26                        Land   
                           27                        Wahl   
                           28                        Ring   
                           29                 Hausaufgabe   
                           30                 Deutschland   
                           31                       Seite   
                           32                         USA   
                           33                        Bild   
                           34                      Israel   
                           35                    Programm   
                           36                     Kollege   
                           37                     Hamburg   
                           38                      Mensch   
                           39                  Mathematik   
                           40                        Kind   
                           41                    Training   
                           42                         Art   
                           43         Nachhilfeunterricht   
                           44                       Organ   
                           45                         Uhr   
                           46                     Sendung   
                           47                        Witz   
                           48                        Abs.   
                           49                Arbeitsplatz   
                           50                      Mensch   
                           51                        Euro   
                           52                    Hochzeit   
                           53                  Österreich   
                           54                      Berlin   
                           55                   Kommentar   
                           56               Veranstaltung   
                           57                     Störung   
                           58                    Akademie   
                           59                         Tag   
                           60                  Ausbildung   
                           61                    Fahrzeug   
                           62                   Regierung   
                           63                     Beitrag   
                          

-----

### create gold standard for reranking

In [60]:
tpx.stack()

dataset  param_id  nb_topics  topic_idx        
dewac1   e42       100        0          term0                         Partei
                                         term1                            PDS
                                         term2                       Arbeiter
                                         term3                          Kampf
                                         term4                    Sozialismus
                                         term5                            DDR
                                         term6                      Kommunist
                                         term7                   Gewerkschaft
                                         term8                   Kapitalismus
                                         term9                          Linke
                                         term10                      Bewegung
                                         term11                       Genosse
                

In [10]:
res.drop(bad_raters, axis=1)

,,,4,5,6,7,9,11,13,14,15,16
,,topic,,,,,,,,,,
T001_01,Topic 1: Partei,T001,False,False,False,False,False,False,False,False,False,False
T001_02,Topic 1: PDS,T001,False,False,False,False,False,False,False,False,False,False
T001_03,Topic 1: Arbeiter,T001,False,False,False,False,False,False,False,False,False,False
T001_04,Topic 1: Kampf,T001,True,False,False,False,False,False,False,False,True,False
T001_05,Topic 1: Sozialismus,T001,False,False,False,False,False,False,False,False,False,False
T001_06,Topic 1: DDR,T001,False,False,False,False,False,False,False,False,False,False
T001_07,Topic 1: Kommunisten,T001,False,False,False,False,False,False,False,False,False,False
T001_08,Topic 1: Gewerkschaften,T001,False,False,False,False,False,False,False,False,False,False
T001_09,Topic 1: Kapitalismus,T001,False,False,False,False,False,False,False,False,False,False


In [76]:
def rank_top_10(grp):
    grp = grp.reset_index()
    grp['selected'] = grp[[4, 5, 6, 7, 9, 11, 13, 14, 15, 16]].sum(axis=1)
    grp = grp.sort_values('selected', kind='mergesort')
    grp = grp.head(10)
    grp = grp.reset_index().rename(columns={'index': 'old_index'})
    grp = grp.sort_index()
    return grp
    

rankings = res.drop(bad_raters, axis=1).copy()
rankings.insert(0, column='lemma', value=tpx.stack().values)
rankings = rankings.groupby('topic').apply(rank_top_10).drop(['level_0', 'topic'], axis=1).rename(columns={'level_1': 'presented_term'})
tpat = re.compile(r'^Topic \d+: ')
rankings.presented_term = rankings.presented_term.apply(lambda x: re.sub(r'^Topic \d+: ', '', x).strip())

rankings

old_index              presented_term                       lemma  \
topic                                                                        
T001  0          0                      Partei                      Partei   
      1          1                         PDS                         PDS   
      2          2                    Arbeiter                    Arbeiter   
      3          4                 Sozialismus                 Sozialismus   
      4          5                         DDR                         DDR   
      5          6                 Kommunisten                   Kommunist   
      6          7              Gewerkschaften                Gewerkschaft   
      7          8                Kapitalismus                Kapitalismus   
      8          9                       Linke                       Linke   
      9         11                    Genossen                     Genosse   
T002  0          0                       König                       König   
      1          1                        Frau                        Frau   
      2          2                     Mädchen                     Mädchen   
      3          3                        Mann                        Mann   
      4          9                       Vater                       Vater   
      5         10                      Bruder                      Bruder   
      6         12                      Kinder                        Kind   
      7         14                      Mutter                      Mutter   
      8         16                     Tochter                     Tochter   
      9         17                   Schwester                   Schwester   
T003  0          0         Lebenspartnerschaft         Lebenspartnerschaft   
      1          1               Lebenspartner               Lebenspartner   
      2          2                         Ehe                         Ehe   
      3          4                     Partner                     Partner   
      4          6                   Ehegatten                    Ehegatte   
      5          8                       Paare                        Paar   
      6         11               Partnerschaft               Partnerschaft   
      7         13          Lebensgemeinschaft          Lebensgemeinschaft   
      8         14  Lebenspartnerschaftsgesetz  Lebenspartnerschaftsgesetz   
      9         17                Homosexuelle                Homosexuelle   
T004  0          1                 Betriebsrat                 Betriebsrat   
      1          4                     Betrieb                     Betrieb   
      2          5                 Arbeitgeber                 Arbeitgeber   
      3          9               Arbeitgeberin               Arbeitgeberin   
      4         10         Mitbestimmungsrecht         Mitbestimmungsrecht   
      5         12                Arbeitnehmer                Arbeitnehmer   
      6          7        Betriebsvereinbarung        Betriebsvereinbarung   
      7         13        Landesarbeitsgericht        Landesarbeitsgericht   
      8         14             Einigungsstelle             Einigungsstelle   
      9         17                       Senat                       Senat   
T005  0          1                Aufsichtsrat                Aufsichtsrat   
      1          2                 Unternehmen                 Unternehmen   
      2          3                   Aktionäre                    Aktionär   
      3          4                    Vorstand                    Vorstand   
      4          5                      Aktien                       Aktie   
      5         10                Grundkapital                Grundkapital   
      6         12          Aktiengesellschaft          Aktiengesellschaft   
      7         14     Aufsichtsratsmitglieder       Aufsichtsratsmitglied   
      8          0                Gesellschaft                Gesellschaft   
      9          6            H

In [77]:
rankings_reshaped = rankings.lemma.unstack()
rankings_reshaped.index = tpx.index.copy()
rerank_cols = [f'term{i}' for i in range(10)]
rankings_reshaped.columns = rerank_cols
rankings_reshaped

term0  \
dataset param_id nb_topics topic_idx                           
dewac1  e42      100       0                          Partei   
                           1                           König   
                           2             Lebenspartnerschaft   
                           3                     Betriebsrat   
                           4                    Aufsichtsrat   
                           5                      Verfassung   
                           6                          Kirche   
                           7                       Verfahren   
                           8                         Richter   
                           9                     Zivildienst   
                           10                    Information   
                           11                  Studiengebühr   
                           12                          Musik   
                           13                         Winter   
                           14                    Unternehmen   
                           15                            Jan   
                           16                            SPD   
                           17                   Rechtsanwalt   
                           18                           Zeit   
                           19                       Regelung   
                           20                      Bewerbung   
                           21                          Spiel   
                           22                           Frau   
                           23                     Bibliothek   
                           24                           Tier   
                           25                           Gott   
                           26                           Land   
                           27                           Wahl   
                           28                         Müller   
                           29                    Hausaufgabe   
                           30                    Deutschland   
                           31                       Internet   
                           32                            USA   
                           33                          Licht   
                           34                         Israel   
                           35                       Programm   
                           36                        Kollege   
                           37                        Hamburg   
                           38                         Denken   
                           39                     Mathematik   
                           40                           Kind   
                           41                           Held   
                           42                         Kanton   
                           43                       Englisch   
                           44                    Ministerrat   
                           45                        Freitag   
                           46         deutsch_Tierschutzbund   
                           47                           Bank   
                           48                         Steuer   
                           49                   Arbeitsplatz   
                           50                          Staat   
                           51                         Körper   
                           52                       Zauberer   
                           53                           Wien   
                           54                          Reich   
                           55                     Kommentier   
                           56                  Veranstaltung   
                           57                        Störung   
                           58                        Potsdam   
                           59                            Tag   
                           60                        Studium   
                       

In [78]:
reranks = load('dewac1', 'rerank', 100, 'e42')
reranks

Reading ../data/preprocessed/LDAmodel/noun/bow/topics/dewac1_reranker-candidates.csv


term0  \
dataset param_id nb_topics topic_idx metric                                  
dewac1  e42      100       0         c_npmi                         Partei   
                                     c_uci                          Partei   
                                     c_v                               PDS   
                                     d2v_drank                         PDS   
                                     d2v_matches                       PDS   
                                     d2v_rrank                      Partei   
                                     df_drank                       Partei   
                                     df_rrank                       Partei   
                                     drank_vote_vec                 Partei   
                                     dw_drank                       Partei   
                                     dw_rrank                       Partei   
                                     dwf_drank                      Partei   
                                     dwf_rrank                      Partei   
                                     ftx_drank                      Partei   
                                     ftx_matches                  Arbeiter   
                                     ftx_rrank                      Partei   
                                     matches_vote_vec             Arbeiter   
                                     ref                            Partei   
                                     rrank_vote_vec                 Partei   
                                     u_mass                         Partei   
                                     vote_coh                       Partei   
                                     vote_vec                       Partei   
                                     w2v_drank                      Partei   
                                     w2v_matches                  Arbeiter   
                                     w2v_rrank                      Partei   
                                     wf_drank                       Partei   
                                     wf_rrank                       Partei   
                           1         c_npmi                        Mädchen   
                                     c_uci                           König   
                                     c_v                           Mädchen   
                                     d2v_drank                        Frau   
                                     d2v_matches                      Frau   
                                     d2v_rrank                        Frau   
                                     df_drank                         Frau   
                                     df_rrank                         Frau   
                                     drank_vote_vec                   Frau   
                                     dw_drank                         Frau   
                                     dw_rrank                         Frau   
                                     dwf_drank                        Frau   
                                     dwf_rrank                        Frau   
                                     ftx_drank                        Frau   
                                     ftx_matches                      Frau   
                                     ftx_rrank                        Frau   
                                     matches_vote_vec                 Frau   
                                     ref                             König   
                                     rrank_vote_vec                   Frau   
                                     u_mass                          König   
                                     vote_coh                        König   
                                     vote_vec                         Frau   
                                     w2v_drank                        Frau   
             

In [79]:
rankings_reshaped.iloc[0]

term0          Partei
term1             PDS
term2        Arbeiter
term3     Sozialismus
term4             DDR
term5       Kommunist
term6    Gewerkschaft
term7    Kapitalismus
term8           Linke
term9         Genosse
Name: (dewac1, e42, 100, 0), dtype: object

In [80]:

jaccard_similarity_score(y_true=rankings_reshaped.iloc[0].values, y_pred=rankings_reshaped.iloc[0].values)

1.0

In [87]:
np.set_printoptions(linewidth=800)

def jaccard_index(ser):
    gold = rankings_reshaped.loc[ser.name[:-1], rerank_cols]
    intersection = np.intersect1d(ser, gold)
    union = np.union1d(ser, gold)
    return len(intersection) / len(union)

reranks['jaccard'] = reranks[rerank_cols].apply(jaccard_index, axis=1)
reranks

term0  \
dataset param_id nb_topics topic_idx metric                                  
dewac1  e42      100       0         c_npmi                         Partei   
                                     c_uci                          Partei   
                                     c_v                               PDS   
                                     d2v_drank                         PDS   
                                     d2v_matches                       PDS   
                                     d2v_rrank                      Partei   
                                     df_drank                       Partei   
                                     df_rrank                       Partei   
                                     drank_vote_vec                 Partei   
                                     dw_drank                       Partei   
                                     dw_rrank                       Partei   
                                     dwf_drank                      Partei   
                                     dwf_rrank                      Partei   
                                     ftx_drank                      Partei   
                                     ftx_matches                  Arbeiter   
                                     ftx_rrank                      Partei   
                                     matches_vote_vec             Arbeiter   
                                     ref                            Partei   
                                     rrank_vote_vec                 Partei   
                                     u_mass                         Partei   
                                     vote_coh                       Partei   
                                     vote_vec                       Partei   
                                     w2v_drank                      Partei   
                                     w2v_matches                  Arbeiter   
                                     w2v_rrank                      Partei   
                                     wf_drank                       Partei   
                                     wf_rrank                       Partei   
                           1         c_npmi                        Mädchen   
                                     c_uci                           König   
                                     c_v                           Mädchen   
                                     d2v_drank                        Frau   
                                     d2v_matches                      Frau   
                                     d2v_rrank                        Frau   
                                     df_drank                         Frau   
                                     df_rrank                         Frau   
                                     drank_vote_vec                   Frau   
                                     dw_drank                         Frau   
                                     dw_rrank                         Frau   
                                     dwf_drank                        Frau   
                                     dwf_rrank                        Frau   
                                     ftx_drank                        Frau   
                                     ftx_matches                      Frau   
                                     ftx_rrank                        Frau   
                                     matches_vote_vec                 Frau   
                                     ref                             König   
                                     rrank_vote_vec                   Frau   
                                     u_mass                          König   
                                     vote_coh                        König   
                                     vote_vec                         Frau   
                                     w2v_drank                        Frau   
             

In [90]:
reranks.jaccard.unstack('metric').mean().sort_values(ascending=False)

metric
matches_vote_vec    0.602123
w2v_matches         0.599204
vote_vec            0.589772
ftx_matches         0.585645
w2v_drank           0.584138
drank_vote_vec      0.580692
dwf_drank           0.579670
wf_drank            0.579545
dw_drank            0.575646
ftx_drank           0.571143
wf_rrank            0.557137
df_drank            0.556362
rrank_vote_vec      0.553569
w2v_rrank           0.553201
dwf_rrank           0.551550
vote_coh            0.548205
ftx_rrank           0.542864
dw_rrank            0.533401
df_rrank            0.533272
c_npmi              0.530103
c_v                 0.522426
c_uci               0.520709
ref                 0.498485
d2v_drank           0.488223
d2v_matches         0.474886
d2v_rrank           0.462601
u_mass              0.354244
dtype: float64